# VacationPy


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load cities csv
filepath = "../output_data/cities.csv"
cities_df = pd.read_csv(filepath)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = cities_df[["Latitude","Longitude"]]
humidity = cities_df["Humidity (%)"].astype(float)

figure_layout = {
    'width': '550px',
    'height': '350px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add heatmap layer to map
fig = gmaps.figure(layout=figure_layout, zoom_level=1, center=(10,25))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=200, point_radius = 8)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='350px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Ideal cities dataframe created from the following conditions
ideal_cities_df = cities_df.loc[(cities_df["Temperature (C)"] > 21) &
                                 (cities_df["Temperature (C)"] < 28) &
                                 (cities_df["Humidity (%)"] < 50) &
                                 (cities_df["Cloudiness (%)"] < 5) &
                                 (cities_df["Wind Speed (m/s)"] < 4), :]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Dataframe renamed
hotel_df = ideal_cities_df

# Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Parameters dictionary, set to find hotels within 5000m
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# Empty hotels list created for hotel names to be added to 
hotels = []

# Iterrows used to iterate through dataframe, adding first hotel found to hotel names list
for index, row in hotel_df.iterrows():
    coordinates = f"{row['Latitude']}, {row['Longitude']}"
    params['location'] = coordinates
# Try / execpt added as many cities did not return a hotel
    try:
        hotel_data = requests.get(base_url, params=params).json()
        hotels.append(hotel_data["results"][0]["name"])
    except:
        hotels.append(f"No hotel found")

# List of hotel names added to dataframe, removed rows without hotels
hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found", :]
hotel_df

/Users/matthewford/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Latitude,Longitude,Temperature (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date Time,Hotel Name
66,Blackfoot,43.1905,-112.3450,21.54,17,0,2.24,US,2022-08-09 06:25:54,Super 8 by Wyndham Blackfoot/Idaho Falls
69,Cedar City,37.6775,-113.0619,22.54,28,0,2.57,US,2022-08-09 06:24:32,Courtyard by Marriott Cedar City
102,Glendive,47.1053,-104.7125,24.54,43,0,3.09,US,2022-08-09 06:28:15,"Holiday Inn Express & Suites Glendive, an IHG ..."
177,Mount Isa,-20.7333,139.5000,25.87,13,1,2.57,AU,2022-08-09 06:29:21,Redearth Boutique Hotel
275,Chipinge,-20.1883,32.6236,22.42,16,0,1.02,ZW,2022-08-09 06:30:50,Siketaal lodge
480,Abalak,58.1286,68.5944,27.06,43,0,1.84,RU,2022-08-09 06:33:42,Туристический комплекс Абалак


In [10]:
# Used provided template to create hotel info box
info_box_template ="""
<dl>
<dt><b>Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [7]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout=figure_layout, zoom_level=1, center=(10,25))

locations_hotels = hotel_df[["Latitude", "Longitude"]]

marker_layer = gmaps.marker_layer(locations_hotels, info_box_content=hotel_info, display_info_box=True)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='350px', margin='0 auto 0 auto', padding='1px', wi…